In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/MyDrive/Colab Notebooks/KGEmb")
!ls
!echo $(pwd)/data

Mounted at /content/drive
 application.log	 'lap_init FB237 500.npy'    optimizers
 colab_run.ipynb	  Lap_init.ipynb	     requirements.txt
 data			 'lap_init WN18RR 32.npy'    run.py
 data_loader.py		 'lap_init WN18RR 500.npy'   set_env.sh
 datasets		  Lap_No_Rships.ipynb	     test.py
 Lap_Emb.ipynb		  logs			     utils
'lap_init FB237 32.npy'   models
/content/drive/MyDrive/Colab Notebooks/KGEmb/data


In [ ]:
import numpy as np
from scipy import sparse
from sklearn.manifold import SpectralEmbedding
from scipy.spatial.distance import cdist
! pip install pyamg
! pip install fastlapmap
! pip install nmslib
from fastlapmap import LapEigenmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 15.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.5 MB 3.9 MB/s 
     |████████████████████████████████| 188 kB 67.5 MB/s 


In [ ]:
entities=[] # create list to store vertex IDs

with open('data/WN18RR/train.txt', 'r') as f:
    for line in f:
        edge = line.split()
        edge.remove(edge[1]) # remove relationship type
        if int(edge[0]) not in entities:
            entities.append(int(edge[0])) # add head ID
        if int(edge[1]) not in entities:
            entities.append(int(edge[1])) # add tail ID
    f.close()

#This ^^ is to get the number of entities 

mat = sparse.lil_matrix((max(entities),max(entities)), dtype=np.int8).tocsr()

with open('data/WN18RR/train.txt', 'r') as f:
    for line in f:
        edge = line.split()
        row_idx = entities.index(int(edge[0])) # get index for head entity
        col_idx = entities.index(int(edge[2])) # get index for tail entity
        mat[row_idx, col_idx] = 1 # add edge in adjacency matrix
        mat[col_idx, row_idx] = 1 # undirected graph so symetric adjacency matrix
        
embs = LapEigenmap(mat, n_eigs=10, similarity='diffusion', norm_laplacian=False, k=10)

embs = np.asarray(embs)

np.save(embs)

def hit_at_k(r, k): #Function for measuring hit at k for each query, i.e 0 or 1
    if r <= k:
        return 1
    else:
        return 0
    
    
def evaluate(embeddings, head, tail): #Function to calculate metrics for each query
    dist = cdist(embeddings[head,:].reshape(1, -1), embeddings[:,:]).argsort()
    # dist = np.sum((embeddings[rship] - embs[rship,head,:])**2, axis=1).argsort()
    rank = np.where(dist==tail)[1][0]
    recip_rank = 1/rank
    H_1 = hit_at_k(rank, 1)
    H_3 = hit_at_k(rank, 3)
    H_10 = hit_at_k(rank, 10)
    return rank, recip_rank, H_1, H_3, H_10
    
#counter functions to sum across loop
sum_rank = 0
sum_recip_rank = 0
sum_H_1 = 0
sum_H_3 = 0
sum_H_10 = 0
len_test = 0


with open('data/WN18RR/test.txt', 'r') as f:
    for line in f:
        edge = line.split()
        rank, recip_rank, H_1, H_3, H_10 = evaluate(embs, int(edge[0]), int(edge[2]))
        sum_rank += rank
        sum_recip_rank += recip_rank
        sum_H_1 += H_1
        sum_H_3 += H_3
        sum_H_10 += H_10
        
        len_test= len_test + 1
    f.close()
    
#Get the mean/average for metrics
MR = sum_rank / len_test
MRR = sum_recip_rank / len_test
H_1 = sum_H_1 / len_test
H_3 = sum_H_3 / len_test
H_10 = float(sum_H_10) / len_test

#Print results
print('MRR:', MRR, ', Hits@10:', H_10, ', Hits@3: ', H_3, ', Hits@1: ', H_1)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


TypeError: ignored

In [8]:
import numpy as np
from scipy.spatial.distance import cdist
embs = np.load('lap_init WN18RR 32.npy')

(40943, 32)

In [10]:
def hit_at_k(r, k): #Function for measuring hit at k for each query, i.e 0 or 1
    if r <= k:
        return 1
    else:
        return 0
    
    
def evaluate(embeddings, head, tail): #Function to calculate metrics for each query
    dist = cdist(embeddings[head,:].reshape(1, -1), embeddings[:,:]).argsort()
    # dist = np.sum((embeddings[rship] - embs[rship,head,:])**2, axis=1).argsort()
    rank = np.where(dist==tail)[1][0]
    recip_rank = 1/rank
    H_1 = hit_at_k(rank, 1)
    H_3 = hit_at_k(rank, 3)
    H_10 = hit_at_k(rank, 10)
    return rank, recip_rank, H_1, H_3, H_10
    
#counter functions to sum across loop
sum_rank = 0
sum_recip_rank = 0
sum_H_1 = 0
sum_H_3 = 0
sum_H_10 = 0
len_test = 0


with open('data/WN18RR/test.txt', 'r') as f:
    for line in f:
        edge = line.split()
        rank, recip_rank, H_1, H_3, H_10 = evaluate(embs, int(edge[0]), int(edge[2]))
        sum_rank += rank
        sum_recip_rank += recip_rank
        sum_H_1 += H_1
        sum_H_3 += H_3
        sum_H_10 += H_10
        
        len_test= len_test + 1
    f.close()
    
#Get the mean/average for metrics
MR = sum_rank / len_test
MRR = sum_recip_rank / len_test
H_1 = sum_H_1 / len_test
H_3 = sum_H_3 / len_test
H_10 = float(sum_H_10) / len_test

#Print results
print('MRR:', MRR, ', Hits@10:', H_10, ', Hits@3: ', H_3, ', Hits@1: ', H_1)

MRR: 0.00021408112413103257 , Hits@10: 0.0 , Hits@3:  0.0 , Hits@1:  0.0
